## Légifrance data

### Preprocess data 

In [1]:
## I will use pylegifrance library : https://github.com/rdassignies/pylegifrance/?tab=readme-ov-file

In [13]:
import os, config2py
os.environ["LEGIFRANCE_CLIENT_ID"] = config2py.config_getter("LEGIFRANCE_CLIENT_ID")
os.environ["LEGIFRANCE_CLIENT_SECRET"] = config2py.config_getter("LEGIFRANCE_CLIENT_SECRET")

In [21]:
from pylegifrance import LegiHandler
client = LegiHandler()
client.set_api_keys(legifrance_api_key=config2py.config_getter("LEGIFRANCE_CLIENT_ID"), legifrance_api_secret=config2py.config_getter("LEGIFRANCE_CLIENT_SECRET"))

2024-02-20 14:35:18,794 - root - INFO - Connexion à l'api legifrance réussie.


In [22]:
from pylegifrance import recherche_CODE

doc_integrale_travail = recherche_CODE(code_name="Code du travail")

2024-02-20 14:35:49,649 - pylegifrance.process.processors - INFO - Nombre de résultats trouvés: 1


In [23]:
len(doc_integrale_travail[0])

41

In [24]:
doc_integrale_travail[0].keys()

dict_keys(['executionTime', 'dereferenced', 'id', 'idConteneur', 'cid', 'title', 'nor', 'eli', 'alias', 'jorfText', 'jurisState', 'visa', 'modifDate', 'jurisDate', 'dateDebutVersion', 'dateFinVersion', 'signers', 'prepWork', 'dateParution', 'dateTexte', 'numParution', 'notice', 'nota', 'inap', 'textNumber', 'textAbroge', 'etat', 'dossiersLegislatifs', 'nature', 'resume', 'rectificatif', 'motsCles', 'appellations', 'liens', 'observations', 'sections', 'articles', 'pagePdf', 'fileName', 'fileSize', 'filePath'])

In [25]:
## Voir le contenu de doc_integrale
import json

with open('code_travail.json', 'w', encoding='utf-8') as fichier:
    json.dump(doc_integrale_travail, fichier, ensure_ascii=False, indent=4)

In [26]:
def extraire_infos(element, infos=[]):
    if isinstance(element, dict):
        if 'pathTitle' in element and 'content' in element:
            infos.append({
                'pathTitle': element['pathTitle'],
                'content': element['content']
            })

        for key, value in element.items():
            extraire_infos(value, infos)

    elif isinstance(element, list):
        for item in element:
            extraire_infos(item, infos)

    return infos

In [27]:
infos_extraites_code_travail = extraire_infos(doc_integrale_travail[0]['sections'])

In [28]:
len(infos_extraites_code_travail)

11520

In [ ]:
for info in infos_extraites_code_travail:
    print(info)

In [30]:
## pour sos oxygène, c'est que les données de santé, sécurité sociale, ....
code_sante = recherche_CODE(code_name="Code de la santé publique")

2024-02-20 14:36:30,611 - pylegifrance.process.processors - INFO - Nombre de résultats trouvés: 1


In [ ]:
infos_extraites_code_sante = extraire_infos(code_sante[0]['sections'])
infos_extraites_code_sante

In [33]:
len(infos_extraites_code_sante)

25819

In [34]:
with open('code_sante.json', 'w', encoding='utf-8') as fichier:
    json.dump(infos_extraites_code_sante, fichier, ensure_ascii=False, indent=4)

## Create dict with content in text format

In [35]:
infos_extraites_code_sante[0].keys()

dict_keys(['pathTitle', 'content'])

In [36]:
## The content is in html format, so I convert it to text using utf8 encoding and I ignore links as they have not a meaning

import html2text

def convert_content(content, ig_links=True):
    h = html2text.HTML2Text()
    h.ignore_links = ig_links
    if isinstance(content, bytes):
        content = content.decode('utf-8')
    return h.handle(content)

converted_infos = []

for item in infos_extraites_code_sante:
    converted_item = {
        'pathTitle': item['pathTitle'],
        'content': convert_content(item['content'])
    }
    converted_infos.append(converted_item)

In [37]:
len(converted_infos)

25819

In [46]:
from typing import Iterable
from lkj import chunker
import oa


def pathTitle_list_to_text(metadata: list, sep='\n'):
    return sep.join(metadata['pathTitle'])

def pathContent_metadata_to_text(metadata: dict, *, sep='\n'):
    return f"{sep.join(metadata['pathTitle'])}{sep}{metadata['content']}"

In [51]:
from typing import List, Tuple
import tiktoken

def tokenize(text: str, encoding_name : str ) -> List[str]:
    encoding = tiktoken.get_encoding(encoding_name)
    int_tokens = encoding.encode(text)
    str_tokens = [encoding.decode_single_token_bytes(token) for token in int_tokens]
    return str_tokens

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [ ]:
from lkj import chunker
encoding_name = "cl100k_base" 

def process_data(data: dict, max_tokens: int) -> Tuple[str, dict, bool]:
    code_text = pathTitle_metadata_to_text(data)
    # joke_tokens = tokenize(joke_text)
    # joke_tokens = remove_stop_words(joke_tokens)
    joke_tokens = num_tokens_from_string(joke_text, encoding_name)

    if joke_tokens > max_tokens:
        # joke_tokens = joke_tokens[:max_tokens]
        return "", {}, True  # Skips the joke

    # truncated_text = ' '.join(joke_tokens)
    # return truncated_text
    metadata = {
        "source": "Reddit",
        "id": joke['id'],
        "score": joke['score']
    }
    return joke_text, metadata, False

# def process_jokes(jokes: List[dict], max_tokens: int) -> List[Tuple[str, dict]]:
#     processed_jokes = map(lambda joke: process_joke(joke, max_tokens), jokes)
#     return [(joke_text, metadata)  for joke_text, metadata, skip in processed_jokes if not skip]